In [1]:
import openai
import pandas as pd
import spacy
nlp = spacy.load("en_core_web_lg")
import statistics
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
import numpy as np
import collections
from collections import Counter
import re
import language_tool_python  
from nltk.tokenize import sent_tokenize
from gingerit.gingerit import GingerIt
import pprint
from spacy import displacy
from sklearn.svm import SVC
import textstat
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.metrics import BigramAssocMeasures
import math
from nltk.sentiment import SentimentIntensityAnalyzer
'''import sys
sys.path.append("../my_lib/my_func1.py")
from my_lib.my_func1 import *'''

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


'import sys\nsys.path.append("../my_lib/my_func1.py")\nfrom my_lib.my_func1 import *'

In [2]:
def add_feature(df, column_index, feature):
    # Get the column names
    columns = df.columns.tolist()

    # Insert the new feature at the specified index
    columns.insert(column_index, feature)

    # Reindex the dataframe with the updated column order
    df = df.reindex(columns=columns)

    return df

def walk_tree1(node, depth):
    if node.n_lefts + node.n_rights > 0:
        dep=[]
        d=0
        for child in node.children:
            d=walk_tree1(child, depth + 1)
            dep.append(d)
        return max(dep)
    
    else:
        return depth
    
def avg_sentence_complexity(str1):
    final_list=[]
    nlp = spacy.load("en_core_web_lg")
    temp_list=[]
    sents=[]
    txt = str1
    txt=" ".join(txt.split())
    text = nlp(txt)
    sent_tokenize1 = (list(text.sents))
    sents.append(sent_tokenize1[0])
    for sent in text.sents:
        d=walk_tree1(sent.root, 0)
        temp_list.append(d)
            #displacy.render(sent, style="dep")
        final_list.append(temp_list)
        #print("essay first line ",sents[len(sents)-1],"max is ",max(temp_list)," min is ",min(temp_list)," mean is ", round(statistics.mean(temp_list),3),
        #  " median is ",statistics.median(temp_list)," mode is ",statistics.mode(temp_list))
    return round(statistics.mean(temp_list),3)

def grammar_check(str1):

    my_tool = language_tool_python.LanguageTool('en-GB')###########
    matches = my_tool.check(str1)
    list_s1=[]
    if(matches):
        fs=str1.split(".")[0]
        #print(i," index | essay first sentences ",fs)
        for match in matches:
            print(match)
    #print(match.ruleId)
    #print('Message:',match.message,"\n")
            #print("####################### ",match.context)
            if match.replacements:
                chopped_string = match.replacements[0].replace(',','').strip()
                list_s1.append(chopped_string)
            else:
                list_s1.append(match.ruleId)    
        #print("final list",list_s1)
    return len(list_s1)

def generate_ngrams(s, n):
    # Convert to lowercases
    s = s.lower()
    
    # Replace all none alphanumeric characters with spaces
    s = re.sub(r'[^a-zA-Z0-9\s]', ' ', s)
    
    # Break sentence in the token, remove empty tokens
    tokens = [token for token in s.split(" ") if token != ""]
    
    # Use the zip function to help us generate n-grams
    # Concatentate the tokens into ngrams and return
    ngrams = zip(*[tokens[i:] for i in range(n)])
    list1=[" ".join(ngram) for ngram in ngrams]
    #return [" ".join(ngram) for ngram in ngrams]
    return list1

def rep_count(str1):
    tot_rp=[]
    max_rp=[]
    #print(ess[i],"\n","new","\n")
    list1=generate_ngrams(str1, 5)
    myset = set(list1)
    rept = len(list1) - len(myset)
    tot_rp.append(rept)
    c = collections.Counter(list1)
    c.most_common(1)[0][1]
    max_rp.append(c.most_common(1)[0][1])
    #print("\n\nnew essay index ",i,": for essay ",list1[0],":")
    print("repeat ",rept," times | most common phrase is **",c.most_common(1)[0][0], "** |number of repeataion happened: ",c.most_common(1)[0][1]," times\n")
    #print("\ncommon phrases: \n",c)
    #print("\nmy set: \n",myset)
    #print("list ",tot_rp," ",max_rp)
    #print("individual ",rept," ",c.most_common(1)[0][1])
    return rept,c.most_common(1)[0][1],c.most_common(10)

def most_frequent(str1):

    nlp = spacy.load("en_core_web_lg")
    str1=" ".join(str1.split())
    text = nlp(str1)
    words = [token.text for token in text if token.is_stop!= True and token.is_punct!= True]
    w_c=Counter(words)
    m_f=w_c.most_common(1)[0][1]
    
    tags=[]
    for w in text:
        tags.append(w.pos_)
    tag_freq = Counter(tags)
    #print ("=====tag FREQUENCY=====")
    #print(tag_freq.values())

    w_l=word_cnt(str1)
    for key in tag_freq:
        #print("hi")
        if key=='NOUN':
            n=tag_freq[key]/w_l
            #print("n ",n)
        elif key == 'VERB':
            v=tag_freq[key]/w_l
            #print("v ",v)
        elif key =='ADJ':
            a=tag_freq[key]/w_l
            #print("a ",a)
        elif key =='PRON':
            p=tag_freq[key]/w_l

        elif key=='MD':
            print("the number of modal verbs ",key, tag_freq.values())
            #print("p ",p)
        #print("key is ",key," value ",tag_freq[key], " frequency ", tag_freq[key]/w_l)
    return m_f,n,v,a,p

def uniq_w(str1):
    return len(set(str1.split()))

def stop_w(str1):
    nlp = spacy.load("en_core_web_lg")
    text = nlp(str1)
    words = [token.text for token in text if token.is_stop == True]
    #print(words)
    return(len(words))

def stop_w_r(str1):
    number_of_sentences = sent_tokenize(str1)
    n=len(number_of_sentences)

    nlp = spacy.load("en_core_web_lg")
    text = nlp(str1)
    words = [token.text for token in text if token.is_stop == True]
    #print(words)
    
    l=len(words)
    swr=l/n
    return swr

def sen_len(str1):
    number_of_sentences = sent_tokenize(str1)
    i=0
    avg=0
    sl=[]
    for num in number_of_sentences:
        i+=1
        words=num.split()
        avg+=len(words)
        sl.append(len(words))
    #print(num, len(num),len(words))
    avg=avg/i
    md=statistics.mode(sl)
    mx=max(sl)
    mn=min(sl)
    return avg,md,mx,mn

def sen_num(str1):
    number_of_sentences = sent_tokenize(str1)
    n=len(number_of_sentences)
    #print(n)
    return n

def word_cnt(str1):
    words=str1.split()
    return len(words)

def lexical_diversity(str1):
    tokens = word_tokenize(str1)

    lex_div = len(set(tokens)) / len(tokens)
    print("Lexical Diversity Score of essay:", lex_div)
    return lex_div

def discourse_conn(str1):
    tokens = nltk.word_tokenize(str1)

    # Perform part-of-speech tagging
    pos_tags = nltk.pos_tag(tokens)

    # Identify discourse connectors based on part-of-speech patterns
    discourse_connectors = [token for token, pos in pos_tags if pos in ['CC', 'RB', 'IN']]

    # Output the identified discourse connectors
    print("Discourse connectors:", discourse_connectors)
    print("Discourse connector count:", len(discourse_connectors))
    return len(discourse_connectors)

def sentence_similarity(str1):
    text = nlp(str1)
    sent_tokenize = (list(text.sents))
    i=0
    sensim=0.0
    sensim_list=[]
    for sent2 in sent_tokenize:
        i+=1
        if i == 1:
            sent1=sent2
            continue
        #print (sent2.similarity(sent1))
        sensim+=sent2.similarity(sent1)
        sensim_score=sent2.similarity(sent1)
        sent1=sent2
        sensim_list.append(sensim_score)
    sensim=sensim/i
    #print("average sentence similarity is ",sensim)

    return sensim,sensim_list

def word_similarity(str1):
    text = nlp(str1)
    p=0
    ps=0
    n=0
    ns=0
    total=0
    word = [token.text for token in text if token.is_punct != True]
    for i in range(len(word)):
    #x.append(i)
    #print(i)
    #print(words[i])
        w2=word[i]
        if i == 0:
            w1=w2
            continue
    #print (nlp(w2).similarity(nlp(w1)))
        #print(w1+ " "+w2+" ",nlp(w2).similarity(nlp(w1)))
        if nlp(w2).similarity(nlp(w1))>0:
            p=p+1
            ps+=nlp(w2).similarity(nlp(w1))
        elif nlp(w2).similarity(nlp(w1))<0:
            n=n+1
            ns+=nlp(w2).similarity(nlp(w1))
    #y1=y1+float(nlp(w2).similarity(nlp(w1)))
    #print(y1)
    #y.append(y1)
        total+=nlp(w2).similarity(nlp(w1))
        w1=w2
    #print(p,n)
    return p,n,total,ps,ns

def get_modal_verbs(text):
    #nlp = spacy.load('en_core_web_sm')
    doc = nlp(text)
    cnt=0
    modal_verbs = []
    for token in doc:
        if token.pos_ == 'VERB' and token.tag_ == 'MD':
            modal_verbs.append(token.lemma_)
            cnt+=1

    return cnt,modal_verbs

def calculate_readability(text):
    # Calculate the Flesch-Kincaid Grade Level
    flesch_grade = textstat.flesch_kincaid_grade(text)

    # Calculate the Automated Readability Index (ARI)
    ari = textstat.automated_readability_index(text)

    # Calculate the Coleman-Liau Index
    coleman_liau = textstat.coleman_liau_index(text)

    # Return the readability scores
    return flesch_grade, ari, coleman_liau

def calculate_cohesion(text):
    # Tokenize the text
    tokens = word_tokenize(text)

    # Remove stopwords and punctuation
    stop_words = set(stopwords.words("english"))
    tokens = [token.lower() for token in tokens if token.isalnum() and token.lower() not in stop_words]

    # Lemmatize the tokens
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # Convert generator to a list of bigrams
    bigrams = list(nltk.bigrams(tokens))

    # Calculate bigram frequencies
    bigram_freq = Counter(bigrams)

    # Calculate pointwise mutual information (PMI) for bigrams
    total_bigrams = len(bigrams)
    pmi_scores = {}
    for bigram, freq in bigram_freq.items():
        pmi = math.log(freq / total_bigrams / ((tokens.count(bigram[0]) / len(tokens)) * (tokens.count(bigram[1]) / len(tokens))))
        pmi_scores[bigram] = pmi

    # Calculate average PMI as a measure of cohesion
    avg_pmi = sum(pmi_scores.values()) / len(pmi_scores)

    return avg_pmi

def calculate_uniqueness_score(text):
    # Tokenize the text into words
    words = word_tokenize(text)

    # Calculate the frequency distribution of words
    word_freq = Counter(words)

    # Calculate the uniqueness score as the ratio of unique words to total words
    uniqueness_score = len(word_freq) / len(words)

    return uniqueness_score

def detect_emotional_tone(text):
    sid = SentimentIntensityAnalyzer()
    sentiment_scores = sid.polarity_scores(text)
    return sentiment_scores

def para_cnt(str1):
    paragraphs = str1.split("\n\n")
    return len(paragraphs)

def discourse_marker():
    with open('../other/da.txt',"r") as file:
        text=file.read()
    d_list = text.split(",")
    #print(d_list)
    d_list.remove('')
    #index = d_list.index('')
    #print(index)
    #d_list.remove('')
    return d_list

def discourse_marker_check(essay):
    d_list=discourse_marker()
    count=0
    for marker in d_list:
        occurrences = re.findall(r'\b' + re.escape(marker) + r'\b', essay, flags=re.IGNORECASE)
        count += len(occurrences)
        #print("occurances ",occurrences," for marker ",marker)
    #print("count ",count)
    return count

In [51]:
str1="write an ielts essay on "
str2="Protect wild animals"
str3=" don't add titles, and heading of each paragraphs in the essay"
str4=str1+str2+str3
print(str4)
essay_list=[str4]

write an ielts essay on Protect wild animals don't add titles, and heading of each paragraphs in the essay


In [52]:
df = pd.DataFrame()
with open('../other/key1.h', 'r') as file:
 data = file.read()
openai.api_key=data
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
]
for i in range(len(essay_list)):
    message = essay_list[i]

    messages.append(
        {"role": "user", "content": message},
            )
    chat_completion = openai.ChatCompletion.create(
            model="gpt-3.5-turbo", messages=messages
        )
    reply = chat_completion.choices[0].message.content
    #print(reply)
    messages.append({"role": "assistant", "content": reply})
    #substring = "essay on "
    #modified_string = message.replace(substring, "")

    excluded_beginning = "write an ielts essay on "
    excluded_end = " don't add titles, and heading of each paragraphs in the essay"

# Copy the string excluding the specified parts
    modified_string = message[len(excluded_beginning):-len(excluded_end)]

    lines = reply.splitlines()
    if lines[0].startswith('As a helpful assistant, I will provide you with an essay on the topic'):
        print(lines[1:])
        print(lines[2:])
        str1="\n".join(lines[2:])
    elif lines[0].startswith(modified_string):
        str1="\n".join(lines[2:])
    else:
       str1=reply
    df=df.append({'essay':reply,'topic':modified_string},ignore_index=True)
#print("message without essay", message, modified_string)
#print("the actual essay", reply)

/tmp/ipykernel_280715/2088402255.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append({'essay':reply,'topic':modified_string},ignore_index=True)


In [5]:
pd.set_option('display.max_columns', None)
print(df)
print(modified_string)

                                               essay  \
0  Family History and Future Generations\n\nIntro...   

                                               topic  
0  learn about family history or focus on present...  
learn about family history or focus on present and future generation


In [53]:
ess=df.loc[:,'essay']
print(ess[0])

t=df.loc[:,'topic']
print(t[0])

In recent years, the conservation and protection of wild animals have become increasingly significant topics of discussion. It is crucial for humans to recognize the importance of safeguarding these creatures and take necessary measures to ensure their survival. This essay delves into the reasons why protecting wild animals is crucial, the threats they face, and the potential solutions.

Firstly, wild animals play a vital role in maintaining ecological balance. Various species contribute to the delicate balance of their respective ecosystems by regulating populations of other organisms and preserving biodiversity. For instance, predatory animals help regulate the population of herbivores, thereby preventing overgrazing and the consequent degradation of vegetation.

Furthermore, wildlife tourism has emerged as a significant source of income for many countries. Protected areas, such as national parks and wildlife reserves, attract millions of tourists annually. These visitors contribute 

#if first line is title/repeatation of the topic
lines = reply.splitlines()
ess="\n".join(lines[2:])
print(ess)
data.loc[:,'essay']=ess

In [54]:
data=df
data
ess=data.loc[:,'essay']

data.loc[:,'essay']=ess
data.loc[:,'topic']=modified_string

In [105]:
data

,essay,topic
0,Television has become a prevalent form of ente...,TELEVISION DOMINATES THE FREE-TIME OF TOO MANY...


pd.set_option('display.max_columns', None)
#print(data)
print(data.loc[:,'topic'])

In [55]:
for i in range(len(ess)):
    pc=para_cnt(ess[i])
    data.loc[i,'para_count']=pc

    wc=word_cnt(ess[i])
    data.loc[i,'word_count']=wc

    ns=sen_num(ess[i])
    data.loc[i,'sentence_count']=ns

    avg,md,mx,mn=sen_len(ess[i])
    data.loc[i,'sentence_len(mean)']=avg
    data.loc[i,'sentence_len(mode)']=md
    data.loc[i,'sentence_len(max)']=mx
    data.loc[i,'sentence_len(min)']=mn

    sw=stop_w(ess[i])
    swr=stop_w_r(ess[i])
    data.loc[i,'stop_words']=sw
    data.loc[i,'sw_rates']=swr

    uw=uniq_w(ess[i])
    data.loc[i,'unique_words']=uw

    uw=uniq_w(ess[i])
    mf,n,v,a,p=most_frequent(ess[i])
    mvc,mv=get_modal_verbs(ess[i])
    data.loc[i,'noun']=n
    data.loc[i,'verb']=v
    data.loc[i,'adj']=a
    data.loc[i,'pron']=p,
    data.loc[i,'modal verb']=mvc
    data.loc[i,'most_freq']=mf

    tr,mr,rep_list=rep_count(ess[i])
    data.loc[i,'number_of_repeatation']=tr
    data.loc[i,'max_occurance_of_most_frequent_phrase']=mr

    asc=avg_sentence_complexity(ess[i])
    data.loc[i,'avg sentence complexity']=asc

    l_d=lexical_diversity(ess[i])
    data.loc[i,'lexical diversity']=l_d

    sensim,sensim_list=sentence_similarity(ess[i])
    sensim_list.sort()
    sensim_mod=statistics.mode(sensim_list)
    sensim_med=statistics.median(sensim_list)
    data.loc[i,'sentence similarity mean']=sensim
    data.loc[i,'sentence similarity median']=sensim_med
    data.loc[i,'sentence similarity mode']=sensim_mod

    pos,neg,tot,p_s,n_s=word_similarity(ess[i])
    data.loc[i,'word_sim_p_count']=pos
    data.loc[i,'word_sim_p_score']=p_s
    data.loc[i,'word_sim_n_count']=neg
    data.loc[i,'word_sim_n_score']=n_s
    data.loc[i,'word_sim_score']=tot

    f_g, ari, c_l = calculate_readability(ess[i])
    data.loc[i,'flesch grade']=f_g
    data.loc[i,'Automated Readability']=ari
    data.loc[i,'coleman_liau']=c_l

    coh_score=calculate_cohesion(ess[i])
    data.loc[i,'cohesion score']=coh_score

    uniq = calculate_uniqueness_score(ess[i])
    data.loc[i,'uniqueness of words']=uniq

    emot = detect_emotional_tone(ess[i])
    data.loc[i,'positive_sentiment']=emot['pos']
    data.loc[i,'negative_sentiment']=emot['neg']
    data.loc[i,'neutral_sentiment']=emot['neu']

    cnt=discourse_marker_check(ess[i])
    data.loc[i,'discourse_marker']=cnt

    data.loc[i,'label']=1

print(f"The number of columns in the CSV file is: {data.shape[1]}")

repeat  0  times | most common phrase is ** in recent years the conservation ** |number of repeataion happened:  1  times

Lexical Diversity Score of essay: 0.5051334702258727


/tmp/ipykernel_280715/4191345339.py:248: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  if nlp(w2).similarity(nlp(w1))>0:
/tmp/ipykernel_280715/4191345339.py:251: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  elif nlp(w2).similarity(nlp(w1))<0:
/tmp/ipykernel_280715/4191345339.py:257: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  total+=nlp(w2).similarity(nlp(w1))


The number of columns in the CSV file is: 40


In [56]:
data

,essay,topic,para_count,word_count,sentence_count,sentence_len(mean),sentence_len(mode),sentence_len(max),sentence_len(min),stop_words,...,flesch grade,Automated Readability,coleman_liau,cohesion score,uniqueness of words,positive_sentiment,negative_sentiment,neutral_sentiment,discourse_marker,label
0,"In recent years, the conservation and protecti...",Protect wild animals,7.0,429.0,25.0,17.16,20.0,32.0,9.0,168.0,...,13.5,17.0,18.33,4.684587,0.505133,0.161,0.084,0.755,56.0,1.0


In [50]:
data.to_csv('../csv/gpt/csv_gpt3-1.csv',mode='a',header=None,index=False)

In [43]:
data.to_csv('../csv/gpt/csv_gpt3-1_2.csv',mode='a',header=None,index=False)

In [4]:
data = pd.read_csv('../csv/gpt/csv_gpt3-1.csv')
ess=data.loc[:,'essay']

In [ ]:
import language_tool_python

def grammar_check1(text):
# Create separate LanguageTool instances for GB and US English
    tool_gb = language_tool_python.LanguageTool('en-GB')
    tool_us = language_tool_python.LanguageTool('en-US')


# Check for GB English grammar errors
    matches_gb = tool_gb.check(text)
    print("gb ",matches_gb)

# Check for US English grammar errors
    matches_us = tool_us.check(text)
    print("us ",matches_us)

    common_errors = [
    error for error in matches_gb
    if any(error.matches(match) for match in matches_us)
    ]

    print("Errors common to GB and US English:")
    for error in common_errors:
        print(error)

# Filter errors that do not match in either GB or US English
#    non_matching_errors = [
#        error for error in matches_gb + matches_us
#        if not any(error.matches(match) for match in matches_gb + matches_us)
#    ]

#    print("Errors not matching in either GB or US English:")
#    for error in non_matching_errors:
#        print(error)

for i in range(len(ess)):
    grammar_check1(ess[i])